In [1]:
#get phenotype list input
#get the list of diseases associated with those phenotypes
#for each disease:
    #for each phenotype get the tf-idf score
        #get the tf: product of disease and phenotype weight
        #get idf: log of the total diseases over the associated diseases
    #then add up all of those scores
    #output the ratio (sum of listed phenotype scores over the total)
#do this each- get ranking of diseases

In [1]:
import pandas as pd
from py2neo import Graph
from py2neo import Node
from matplotlib import pyplot as plt
import numpy as np

In [13]:
from py2neo import Database
db = Database("bolt://disease.ncats.io:80")
graph = db.default_graph

In [3]:
#get the total tf-idf lists
diseases = pd.read_csv("GARD_TF_IDF.csv")
phenotypes = pd.read_csv("HP_TF_IDF.csv")

In [4]:
total_d = diseases.shape[0]
total_p = diseases.shape[0]

In [5]:
diseases.head()

,Unnamed: 0,ID(n),count(s),n.I_CODE,n.N_Name,Weight,tf_idf_sum
0,0,1,16,"['GARD:0000001', 'OMIM:603358', 'ORPHA:53693',...","['GRACILE SYNDROME', 'FLNMS', 'FINNISH LACTIC ...",0.000046,1.382186e-07
1,1,3,49,"['GARD:0000003', 'OMIM:200110', 'ORPHA:920', '...","['ABLEPHARON MACROSTOMIA SYNDROME', 'AMS', 'CO...",0.000101,1.140221e-06
2,2,7,14,"['GARD:0000005', 'OMIM:200100', 'ORPHA:14', 'O...","['ABETALIPOPROTEINEMIA', 'BASSEN KORNZWEIG SYN...",0.000037,1.303226e-07
3,3,11,29,"['GARD:0000007', 'OMIM:102370', 'ORPHA:969', '...","['ACROMICRIC DYSPLASIA', 'ACROMICRIC SKELETAL ...",0.000060,3.964164e-07
4,4,15,9,"['GARD:0000011', 'OMIM:104290']","['ALTERNATING HEMIPLEGIA OF CHILDHOOD', 'ALTER...",0.000023,7.009057e-08


In [6]:
phenotypes.head()

,Unnamed: 0,ID(n),count(s),d.id,n.I_CODE,n.N_Name,Weight,tf_idf_sum
0,0,82727,5,HP:0001994,"['MESH:D005198', 'SNOMEDCT_US:236468006', 'UML...","['RENAL TUBULAR FANCONI SYNDROME', ""'DE TONI-F...",0.000022,2.729152e-08
1,1,98209,48,HP:0001394,"['UMLS:C0023890', 'MESH:D008103', 'SNOMEDCT_US...","['HEPATIC CIRRHOSIS', 'SCAR TISSUE REPLACES HE...",0.000131,1.109996e-06
2,2,59646,207,HP:0000365,"['HP:0008563', 'HP:0001754', 'HP:0001728', 'HP...","['HEARING DEFECT', 'CONGENITAL DEAFNESS', 'HEA...",0.000470,1.979341e-05
3,3,75348,43,HP:0003128,"['HP:0005960', 'HP:0003255', 'UMLS:C0347959', ...","['LACTICACIDOSIS', 'LACTICACIDEMIA', 'HYPERLAC...",0.000137,9.326714e-07
4,4,78847,2,HP:0012465,"['UMLS:C4022891', 'HP:0012465']","['INCREASED IRON CONCENTRATION IN LIVER', 'INC...",0.000020,5.927394e-09


In [11]:
phenotypes.head(16)

,Unnamed: 0,ID(n),count(s),d.id,n.I_CODE,n.N_Name,Weight,tf_idf_sum
0,0,82727,5,HP:0001994,"['MESH:D005198', 'SNOMEDCT_US:236468006', 'UML...","['RENAL TUBULAR FANCONI SYNDROME', ""'DE TONI-F...",0.000022,2.729152e-08
1,1,98209,48,HP:0001394,"['UMLS:C0023890', 'MESH:D008103', 'SNOMEDCT_US...","['HEPATIC CIRRHOSIS', 'SCAR TISSUE REPLACES HE...",0.000131,1.109996e-06
2,2,59646,207,HP:0000365,"['HP:0008563', 'HP:0001754', 'HP:0001728', 'HP...","['HEARING DEFECT', 'CONGENITAL DEAFNESS', 'HEA...",0.000470,1.979341e-05
3,3,75348,43,HP:0003128,"['HP:0005960', 'HP:0003255', 'UMLS:C0347959', ...","['LACTICACIDOSIS', 'LACTICACIDEMIA', 'HYPERLAC...",0.000137,9.326714e-07
4,4,78847,2,HP:0012465,"['UMLS:C4022891', 'HP:0012465']","['INCREASED IRON CONCENTRATION IN LIVER', 'INC...",0.000020,5.927394e-09
5,5,98159,45,HP:0001397,"['HP:0002252', 'HP:0200121', 'MESH:D005234', '...","['LIVER STEATOSIS', 'FATTY LIVER', 'FATTY INFI...",0.000126,1.005284e-06
6,6,125687,10,HP:0100613,"['UMLS:C4022012', 'HP:0100613']",['DEATH IN EARLY ADULTHOOD'],0.000029,9.115299e-08
7,7,78855,1,HP:0012464,"['UMLS:C0919785', 'HP:0012464']",DECREASED TRANSFERRIN SATURATION,0.000016,1.972279e-09
8,8,111594,3,HP:0004925,"['HP:0004899', 'UMLS:C1839437', 'HP:0004925']",CHRONIC LACTIC ACIDOSIS,0.000024,1.036932e-08
9,9,104771,5,HP:0003452,"['UMLS:C0151900', 'SNOMEDCT_US:165624002', 'HP...",INCREASED SERUM IRON,0.000025,1.753008e-08


In [7]:
#create a function to get the input
def get_disease_ranks(phen_list):
    #phen_list must be a list
    #for now we'll treat these as HP ids
    
    d_sets = []
    for phen in phen_list:
        q = """match (n:S_HP)--(d:DATA) where d.id = {HP_ID} with n match p =(n)-
                    [:R_rel{name: 'has_phenotype'}]-(g:S_GARD) return ID(g);"""
        d = set(graph.run(q, HP_ID = phen).to_series())
        d_sets.append(d)
    
    ids = set.intersection(*d_sets)
    d_df = diseases.loc[diseases['ID(n)'].isin(ids)].copy()
    ratios = []
    for d in ids:
        d_weight = diseases.loc[diseases['ID(n)'] == d, 'Weight'].iloc[0]
        phen_sum = 0
        for phen in phen_list:
            #get the tf-idf
            p_weight = phenotypes.loc[phenotypes['d.id'] == phen, 'Weight'].iloc[0]
            tf = d_weight*p_weight
            associated = phenotypes.loc[phenotypes['d.id'] == phen, 'count(s)'].iloc[0]
            idf = np.log10(total_d/associated)
            tf_idf = tf*idf
            phen_sum += tf_idf
        ratios.append(phen_sum/ diseases.loc[diseases['ID(n)'] == d, 'tf_idf_sum'].iloc[0])
    d_df["rankings"] = ratios
    return d_df

In [44]:
#function that just gives all disease rankings
#does not quite work- it gives a tf-idf score even if the phenotype does not belong to the disease
def get_disease_ranks2(phen_list):
    d_df = diseases.copy()[994:995]
    ratios = []
    for index, row in diseases[994:995].iterrows():
        d_weight = row["Weight"] 
        print(d_weight)
        phen_sum = 0
        for phen in phen_list:
            #get the tf-idf
            p_weight = phenotypes.loc[phenotypes['d.id'] == phen, 'Weight'].iloc[0]
            print(p_weight)
            tf = d_weight*p_weight
            print(tf)
            associated = phenotypes.loc[phenotypes['d.id'] == phen, 'count(s)'].iloc[0]
            print(associated)
            idf = np.log10(total_d/associated)
            print(idf)
            tf_idf = tf*idf
            print(tf_idf)
            phen_sum += tf_idf
            print(phen_sum)
            print("\n")
        ratios.append(phen_sum/ row["tf_idf_sum"])
    d_df["rankings"] = ratios
    return d_df

## Website considerations
- need to run a query to get the overlapping diseases for the phenotype entry
- connect the neo4j database to web app
- connect python to website

In [9]:
get_disease_ranks(phenotypes["d.id"][:5])

,Unnamed: 0,ID(n),count(s),n.I_CODE,n.N_Name,Weight,tf_idf_sum,rankings
0,0,1,16,"['GARD:0000001', 'OMIM:603358', 'ORPHA:53693',...","['GRACILE SYNDROME', 'FLNMS', 'FINNISH LACTIC ...",0.000046,1.382186e-07,0.356445


In [8]:
get_disease_ranks(phenotypes["d.id"][:2]).sort_values(by=["rankings"], ascending = False)

,Unnamed: 0,ID(n),count(s),n.I_CODE,n.N_Name,Weight,tf_idf_sum,rankings
0,0,1,16,"['GARD:0000001', 'OMIM:603358', 'ORPHA:53693',...","['GRACILE SYNDROME', 'FLNMS', 'FINNISH LACTIC ...",0.000046,1.382186e-07,0.093517
403,403,2714,31,"['GARD:0002658', 'OMIM:276700', 'ORPHA:882', '...","['TYROSINEMIA TYPE 1', 'TYROSINEMIA TYPE I', '...",0.000080,4.768549e-07,0.046735


In [10]:
get_disease_ranks(phenotypes["d.id"][:1]).sort_values(by=["rankings"], ascending = False)

,Unnamed: 0,ID(n),count(s),n.I_CODE,n.N_Name,Weight,tf_idf_sum,rankings
13,13,63,36,"['GARD:0000048', 'OMIM:220110', 'UMLS:C0268237']","['CYTOCHROME C OXIDASE DEFICIENCY', 'COMPLEX 4...",0.000078,8.334314e-07,0.019868
493,493,3248,141,"['GARD:0003295', 'OMIM:309000', 'ORPHA:534', '...","['LOWE OCULOCEREBRORENAL SYNDROME', 'OCRL', 'O...",0.000305,8.922160e-06,0.009929
0,0,1,16,"['GARD:0000001', 'OMIM:603358', 'ORPHA:53693',...","['GRACILE SYNDROME', 'FLNMS', 'FINNISH LACTIC ...",0.000046,1.382186e-07,0.006209
1505,1505,9024,51,"['GARD:0009755', 'OMIM:219800']","['NEPHROPATHIC CYSTINOSIS', 'CTNS', 'LYSOSOMAL...",0.000116,1.103330e-06,0.005574
403,403,2714,31,"['GARD:0002658', 'OMIM:276700', 'ORPHA:882', '...","['TYROSINEMIA TYPE 1', 'TYROSINEMIA TYPE I', '...",0.000080,4.768549e-07,0.002021


In [14]:
get_disease_ranks(["HP:0000365","HP:0001319"]).sort_values(by=["rankings"], ascending = False)

,Unnamed: 0,ID(n),count(s),n.I_CODE,n.N_Name,Weight,tf_idf_sum,rankings
197,197,1184,49,"['GARD:0001019', 'OMIM:229200', 'ORPHA:90354',...","['BRITTLE CORNEA SYNDROME', 'EDS VIB (FORMERLY...",0.000103,1.095049e-06,0.224688
1344,1344,8050,24,"['GARD:0008756', 'OMIM:301835', 'UMLS:C0796028']","['ARTS SYNDROME', 'ARTS', 'X-LINKED FATAL ATAX...",0.000048,3.670663e-07,0.088435
771,771,4696,50,"['GARD:0005061', 'OMIM:272200', 'ORPHA:585', '...","['MULTIPLE SULFATASE DEFICIENCY', 'MSD', 'JUVE...",0.000100,1.401906e-06,0.076881
0,0,1,16,"['GARD:0000001', 'OMIM:603358', 'ORPHA:53693',...","['GRACILE SYNDROME', 'FLNMS', 'FINNISH LACTIC ...",0.000046,1.382186e-07,0.062957
1648,1648,9662,83,"['GARD:0010130', 'OMIM:606232', 'ORPHA:48652',...","['22Q13.3 DELETION SYNDROME', 'PHELAN-MCDERMID...",0.000159,2.693412e-06,0.047504
1864,1864,10667,31,"['GARD:0010708', 'OMIM:608027', 'ORPHA:97249',...","['PONTOCEREBELLAR HYPOPLASIA TYPE 3', 'CEREBEL...",0.000057,4.953627e-07,0.039480
948,948,5777,81,"['GARD:0006213', 'OMIM:123450', 'ORPHA:281', '...","['CRI DU CHAT SYNDROME', 'CAT CRY SYNDROME', '...",0.000158,2.895371e-06,0.036436


In [17]:
diseases.sample(25)

,Unnamed: 0,ID(n),count(s),n.I_CODE,n.N_Name,Weight,tf_idf_sum
1641,1641,9633,140,"['GARD:0010109', 'OMIM:122470', 'ORPHA:199', '...","['CORNELIA DE LANGE SYNDROME', 'BRACHMANN DE L...",0.000267,8.322394e-06
1442,1442,8652,9,"['GARD:0009534', 'OMIM:226990']",['CHRONIC ACTIVE EPSTEIN-BARR VIRUS INFECTION'...,0.000024,6.697632e-08
473,473,3150,27,"['GARD:0003163', 'OMIM:245400', 'UMLS:C2931744']",LACTIC ACIDOSIS CONGENITAL INFANTILE,0.000058,5.038908e-07
748,748,4598,22,"['GARD:0004955', 'OMIM:164500']","['SPINOCEREBELLAR ATAXIA 7', 'SCA7', 'SPINOCER...",0.000051,2.718916e-07
866,866,5292,40,"['GARD:0005701', 'OMIM:180500', 'ORPHA:782', '...","['AXENFELD-RIEGER SYNDROME', 'RIEGER SYNDROME'...",0.000086,7.387712e-07
1330,1330,7980,8,"['GARD:0008700', 'OMIM:613982']","['OSTEOGENESIS IMPERFECTA TYPE VI', 'OI TYPE 6...",0.000023,3.700211e-08
358,358,2376,65,"['GARD:0002321', 'OMIM:228520', 'ORPHA:2021', ...",FIBROCHONDROGENESIS,0.000131,1.698619e-06
95,95,548,36,"['GARD:0000383', 'ORPHA:53', 'ORPHANET:53', 'U...","['OSTEOPETROSIS AUTOSOMAL DOMINANT TYPE 2', 'O...",0.000077,6.677161e-07
1607,1607,9444,17,"['GARD:0010001', 'OMIM:214700', 'ORPHA:53689',...","['CONGENITAL CHLORIDE DIARRHEA', 'CLD', 'DIARR...",0.000042,1.546366e-07
804,804,5027,28,"['GARD:0005435', 'OMIM:276900', 'ORPHA:231169'...","['USHER SYNDROME, TYPE 1', 'USH1A', 'USHER SYN...",0.000069,5.058662e-07


In [18]:
diseases.sample(25)

,Unnamed: 0,ID(n),count(s),n.I_CODE,n.N_Name,Weight,tf_idf_sum
1338,1338,8022,9,"['GARD:0008732', 'OMIM:306900', 'UMLS:C0008533']","['HEMOPHILIA B', 'CHRISTMAS DISEASE', 'FACTOR ...",0.000035,5.251708e-08
83,83,468,25,"['GARD:0000336', 'OMIM:270750', 'ORPHA:101003'...","['SPASTIC PARAPLEGIA 23', 'SPG 23', 'SPASTIC P...",0.000056,3.332032e-07
1183,1183,7127,37,"['GARD:0007706', 'OMIM:185300', 'ORPHA:3205', ...","['STURGE-WEBER SYNDROME', 'SWS', 'STURGE WEBER...",0.000087,7.422305e-07
1113,1113,6682,28,"['GARD:0007185', 'OMIM:256700', 'ORPHA:635', '...","['NEUROBLASTOMA', 'NB']",0.000080,5.214520e-07
43,43,173,24,"['GARD:0000134', 'OMIM:184250']",['SPONDYLOEPIMETAPHYSEAL DYSPLASIA STRUDWICK T...,0.000054,2.754413e-07
1667,1667,9764,5,"['GARD:0010198', 'OMIM:609529', 'UMLS:C1836032']","['IMMUNOGLOBULIN A DEFICIENCY 2', 'IGAD2', 'IM...",0.000020,1.447973e-08
887,887,5399,14,"['GARD:0005803', 'OMIM:300068', 'ORPHA:754', '...","['ANDROGEN INSENSITIVITY SYNDROME', 'AIS', 'TE...",0.000039,7.037794e-08
752,752,4624,14,"['GARD:0004976', 'OMIM:613686', 'UMLS:C1852521']","['SPONDYLOCOSTAL DYSOSTOSIS 4', 'SPONDYLOCOSTA...",0.000043,1.112282e-07
647,647,4002,58,"['GARD:0004264', 'OMIM:260565', 'ORPHA:2836', ...","['PEHO SYNDROME', 'PROGRESSIVE ENCEPHALOPATHY ...",0.000120,1.494002e-06
1376,1376,8172,23,"['GARD:0009193', 'OMIM:607684', 'ORPHA:99939',...","['CHARCOT-MARIE-TOOTH DISEASE TYPE 2E', 'CMT 2...",0.000051,2.979734e-07


In [19]:
diseases.sample(25)

,Unnamed: 0,ID(n),count(s),n.I_CODE,n.N_Name,Weight,tf_idf_sum
1549,1549,9225,41,"['GARD:0009870', 'OMIM:231530', 'ORPHA:71212',...",['3-ALPHA HYDROXYACYL-COA DEHYDROGENASE DEFICI...,0.000105,8.292923e-07
244,244,1426,13,"['GARD:0001253', 'OMIM:601382', 'ORPHA:99955',...","['CHARCOT-MARIE-TOOTH DISEASE TYPE 4B1', 'CMT ...",0.000036,1.339881e-07
1460,1460,8802,15,"['GARD:0009630', 'OMIM:602629', 'ORPHA:98806',...","['DYT-THAP1', 'DYT6', 'TORSION DYSTONIA ADULT ...",0.000041,1.128157e-07
2045,2045,11887,22,"['GARD:0012501', 'ORPHA:391372', 'ORPHANET:391...",['INTELLECTUAL DISABILITY-SEVERE SPEECH DELAY-...,0.000046,3.020235e-07
30,30,125,42,"['GARD:0000094', 'UMLS:C0238286', 'ORPHA:578',...","['MUCOLIPIDOSIS TYPE 4', 'ML 4', 'BERMAN SYNDR...",0.000090,1.045035e-06
2060,2060,11937,9,"['GARD:0012539', 'ORPHA:206559', 'ORPHANET:206...","['LIMB-GIRDLE MUSCULAR DYSTROPHY TYPE 2N', 'AU...",0.000025,6.886439e-08
2054,2054,11921,19,"['GARD:0012530', 'ORPHA:55595', 'ORPHANET:5559...","['LIMB-GIRDLE MUSCULAR DYSTROPHY TYPE 1F', 'AU...",0.000047,1.761418e-07
723,723,4453,3,"['GARD:0004777', 'OMIM:262890', 'ORPHA:806', '...","['SCOTT SYNDROME', 'PLATELET FACTOR X RECEPTOR...",0.000015,1.765500e-08
1554,1554,9239,6,"['GARD:0009882', 'OMIM:604931', 'UMLS:C1291245']","['CORTISONE REDUCTASE DEFICIENCY', '11-ALPHA B...",0.000017,2.486804e-08
310,310,2081,34,"['GARD:0001998', 'OMIM:224050', 'ORPHA:1766', ...","['DYSEQUILIBRIUM SYNDROME', 'DES', 'CEREBELLAR...",0.000066,7.267983e-07


In [20]:
diseases.sample(10)

,Unnamed: 0,ID(n),count(s),n.I_CODE,n.N_Name,Weight,tf_idf_sum
297,297,1957,5,"['GARD:0001839', 'OMIM:601410', 'UMLS:C1832386']","['TRANSIENT NEONATAL DIABETES MELLITUS', 'TNDM...",0.000017,1.094438e-08
203,203,1210,19,"['GARD:0001045', 'OMIM:225060', 'UMLS:C2931488']","['BUSTOS SIMOSA PINTO CISTERNAS SYNDROME', 'AU...",0.000049,1.983754e-07
908,908,5525,21,"['GARD:0005926', 'OMIM:605808', 'ORPHA:179', '...","['BIRDSHOT CHORIORETINOPATHY', 'BSCR', 'MULTIP...",0.000073,2.136838e-07
413,413,2826,14,"['GARD:0002816', 'OMIM:243700', 'ORPHA:169446'...","['AUTOSOMAL RECESSIVE HYPER IGE SYNDROME', 'DO...",0.000041,1.190007e-07
485,485,3209,19,"['GARD:0003242', 'OMIM:119500']","['POPLITEAL PTERYGIUM SYNDROME', 'PPS', 'CLEFT...",0.000052,1.853430e-07
1185,1185,7145,2,"['GARD:0007721', 'OMIM:300813', 'ORPHA:3273', ...","['SYNOVIAL SARCOMA', 'SYNOVIAL CELL SARCOMA', ...",0.000014,3.805420e-09
1265,1265,7616,32,"['GARD:0008426', 'OMIM:218700', 'ORPHA:95720',...","['THYROID DYSGENESIS', 'THYROID, ECTOPIC', 'TH...",0.000074,5.485865e-07
932,932,5703,23,"['GARD:0006114', 'OMIM:215700']","['CITRULLINEMIA TYPE I', 'CLASSIC CITRULLINEMI...",0.000056,3.597376e-07
1658,1658,9690,24,"['GARD:0010148', 'OMIM:602588', 'ORPHA:52429',...","['BRANCHIOOTIC SYNDROME', 'BOS1', 'BO SYNDROME...",0.000059,3.230728e-07
1141,1141,6871,27,"['GARD:0007383', 'OMIM:261600', 'ORPHA:716', '...","['PHENYLKETONURIA', 'PKU', 'PHENYLALANINE HYDR...",0.000068,4.365336e-07


In [21]:
diseases.sample(10)

,Unnamed: 0,ID(n),count(s),n.I_CODE,n.N_Name,Weight,tf_idf_sum
780,780,4746,70,"['GARD:0005116', 'OMIM:274000', 'ORPHA:3320', ...","['TAR SYNDROME', 'THROMBOCYTOPENIA ABSENT RADI...",0.000156,2.402339e-06
67,67,377,77,"['GARD:0000273', 'OMIM:278250', 'ORPHA:2834', ...",WRINKLY SKIN SYNDROME,0.000165,2.578938e-06
899,899,5459,56,"['GARD:0005862', 'OMIM:208900', 'ORPHA:100', '...","['ATAXIA TELANGIECTASIA', 'AT', 'LOUIS-BAR SYN...",0.000133,1.718315e-06
354,354,2324,25,"['GARD:0002258', 'UMLS:C1867983', 'ORPHA:99810...","['FAMILIAL PORENCEPHALY', 'FAMILIAL PORENCEPHA...",0.000059,4.232134e-07
1119,1119,6714,10,"['GARD:0007220', 'OMIM:153100', 'ORPHA:79452',...","['MILROY DISEASE', 'HEREDITARY LYMPHEDEMA', 'C...",0.000036,8.745352e-08
236,236,1410,39,"['GARD:0001245', 'OMIM:118220', 'ORPHA:101081'...","['CHARCOT-MARIE-TOOTH DISEASE TYPE 1A', 'CHARC...",0.000091,6.412846e-07
390,390,2628,41,"['GARD:0002553', 'OMIM:114300', 'ORPHA:376', '...","['GORDON SYNDROME', 'ARTHROGRYPOSIS DISTAL TYP...",0.000081,8.525591e-07
901,901,5483,92,"['GARD:0005887', 'OMIM:158350', 'ORPHA:109', '...","['BANNAYAN-RILEY-RUVALCABA SYNDROME', 'BRRS', ...",0.000212,4.384866e-06
727,727,4477,39,"['GARD:0004833', 'OMIM:263520', 'ORPHA:93269',...","['SHORT RIB-POLYDACTYLY SYNDROME, MAJEWSKI TYP...",0.000089,5.887265e-07
1041,1041,6322,70,"['GARD:0006802', 'OMIM:213300', 'ORPHA:475', '...","['JOUBERT SYNDROME', 'CEREBELLOPARENCHYMAL DIS...",0.000148,2.365934e-06


In [26]:
diseases.sort_values(by=["tf_idf_sum"], ascending=False)

,Unnamed: 0,ID(n),count(s),n.I_CODE,n.N_Name,Weight,tf_idf_sum
1214,1214,7305,218,"['GARD:0007891', 'OMIM:194050', 'ORPHA:904', '...","['WILLIAMS SYNDROME', 'WILLIAMS-BEUREN SYNDROM...",0.000452,1.849312e-05
859,859,5266,154,"['GARD:0005683', 'OMIM:270400', 'ORPHA:818', '...","['SMITH-LEMLI-OPITZ SYNDROME', 'SMITH LEMLI OP...",0.000301,1.028139e-05
493,493,3248,141,"['GARD:0003295', 'OMIM:309000', 'ORPHA:534', '...","['LOWE OCULOCEREBRORENAL SYNDROME', 'OCRL', 'O...",0.000305,8.922160e-06
1641,1641,9633,140,"['GARD:0010109', 'OMIM:122470', 'ORPHA:199', '...","['CORNELIA DE LANGE SYNDROME', 'BRACHMANN DE L...",0.000267,8.322394e-06
10,10,39,126,"['GARD:0000029', 'OMIM:214800', 'ORPHA:138', '...","['CHARGE SYNDROME', 'COLOBOMA, HEART ANOMALY, ...",0.000261,7.108356e-06
...,...,...,...,...,...,...,...
894,894,5431,2,"['GARD:0005836', 'OMIM:609135', 'ORPHA:88', 'O...","['APLASTIC ANEMIA', 'ANEMIA APLASTIC', 'APLAST...",0.000010,1.619751e-09
994,994,6047,1,"['GARD:0006485', 'OMIM:137245']","['GASTRIC LYMPHOMA', 'FAMILIAL PRIMARY GASTRIC...",0.000017,1.364585e-09
267,267,1650,1,"['GARD:0001481', 'OMIM:142340', 'ORPHA:2140', ...","['CONGENITAL DIAPHRAGMATIC HERNIA', 'CDH', 'CO...",0.000007,1.250752e-09
1004,1004,6118,1,"['GARD:0006554', 'OMIM:139393', 'ORPHA:2103', ...","['GUILLAIN-BARRE SYNDROME', 'ACUTE AUTOIMMUNE ...",0.000010,9.271684e-10


In [1]:
example = {"dis1" : {1:0.7, 2:0.5, 3:0.5}, "dis2" :{1:.5,4:.6,2:.5}}

In [2]:
example["dis1"][4] = 0.5

In [3]:
example

{'dis1': {1: 0.7, 2: 0.5, 3: 0.5, 4: 0.5}, 'dis2': {1: 0.5, 4: 0.6, 2: 0.5}}

In [6]:
example["dis2"].sort()

AttributeError: 'dict' object has no attribute 'sort'

In [10]:
type(example['dis1'].keys())

dict_keys

In [11]:
set(example['dis1'].keys())

{1, 2, 3, 4}

In [12]:
len(example['dis1'])

4